# Tiktok Ad Scraper

designed to scrape and analyze TikTok ads using the Apify API

In [ ]:
# !pip -q install apify-client tqdm
# !pip -q install tqdm

In [ ]:
from google.colab import drive
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pathlib
from collections import Counter
import os, time, json, re, requests
from tqdm.auto import tqdm
from collections import OrderedDict

os.environ['APIFY_TOKEN'] = ''

from apify_client import ApifyClient
client = ApifyClient(os.environ['APIFY_TOKEN'])

me      = client.user().get()
limits  = client.user().limits()
usage   = client.user().monthly_usage()

In [ ]:
# Uncomment to mount Google Drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/FYP/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/FYP


In [ ]:
from collections import OrderedDict
from tqdm.auto import tqdm
import time
import pandas as pd
import json, pathlib

TARGET_ITEMS = 1000
PAUSE_BETWEEN_RUNS = 2.0
OUT_DIR = pathlib.Path("/content/drive/My Drive/FYP/tiktok_ads")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# QUERIES = [
#     # core identity / community
#     "cat mum", "cat mom", "cat mama", "pet parent", "cat parent", "#catmom", "#catmum",
#     # care / feeding / health
#     "cat care", "kitten care", "cat food", "kitten food", "cat treats", "cat supplements",
#     "litter box", "clumping litter", "natural litter", "automatic litter",
#     "flea treatment", "deworming", "hairball control",
#     # products / enrichment
#     "cat toys", "interactive cat toy", "cat scratcher", "cat tree", "cat tower",
#     "cat water fountain", "cat grooming", "cat brush",
#     # lifestyle / targeting lines
#     "indoor cat", "new kitten", "adopted cat", "rescue cat",
#     "multi cat household", "cat safe", "vet recommended cat",
#     # content hooks advertisers use
#     "for cat moms", "for cat mums", "cat lover gift", "cat owners",
# ]
QUERIES = [
    # Generic ad language
    "sale", "discount", "offer", "deal", "promo", "new collection", "shop now",
    "official store", "limited time", "giveaway", "free shipping",
    # Home / family / pets
    "home decor", "kitchen", "vacuum", "air purifier",
    "baby", "diapers", "stroller",
    "pet", "cat", "dog", "pet food", "litter",
    # Beauty / fashion
    "skincare", "makeup", "lipstick", "foundation", "serum",
    "fashion", "clothing", "shoes", "sneakers", "bags", "accessories",
    # Electronics / gaming
    "smartphone", "laptop", "earbuds", "headphones", "camera",
    "gaming", "console", "pc gaming", "keyboard", "mouse",
    # Food / lifestyle
    "food delivery", "snacks", "coffee", "restaurant", "meal kit",
    "fitness", "supplements", "protein", "yoga", "gym",
    # Services / finance / education / travel
    "insurance", "bank", "credit card", "loan",
    "online course", "bootcamp", "language learning", "edtech",
    "travel", "hotel", "flight", "tour", "luggage",
    # Seasonal / events
    "back to school", "summer sale", "black friday", "cyber monday",
    # Generic character seeds to broaden results space
    "a", "e", "i", "o", "u", "2025", "pro", "max"
]

def run_ads_actor(query: str, country: str = "all"):
    run_input = {"query": query, "country": country}
    return client.actor("lexis-solutions/tiktok-ads-scraper").call(run_input=run_input)

def key_for_dedupe(item: dict) -> str:
    # Prefer video URL, fallback to adId
    return str(item.get("adVideoUrl") or item.get("adId") or json.dumps(item, sort_keys=True))

kept_items = OrderedDict()
progress = tqdm(total=TARGET_ITEMS, desc="Collecting valid ads", unit="ad")

for q in QUERIES:
    if len(kept_items) >= TARGET_ITEMS:
        break

    print(f"\n🔎 Running query: {q}")
    try:
        run = run_ads_actor(q, country="all")
        dataset_id = run["defaultDatasetId"]
        ds = client.dataset(dataset_id)

        for item in ds.iterate_items():
            ad_title = (item.get("adTitle") or "").strip()
            if not ad_title:
                continue

            k = key_for_dedupe(item)
            if k not in kept_items:
                item["sourceQuery"] = q
                kept_items[k] = item
                progress.update(1)

            if len(kept_items) >= TARGET_ITEMS:
                break

    except Exception as e:
        print(f"⚠️ Query '{q}' failed: {e}")

    if len(kept_items) < TARGET_ITEMS:
        time.sleep(PAUSE_BETWEEN_RUNS)

progress.close()
print(f"\n✅ Collected {len(kept_items)} unique, valid ads (non-empty adTitle, no duplicate videos).")

# Convert to DataFrame & save
items = list(kept_items.values())
df = pd.DataFrame(items)

json_path = OUT_DIR / f"tiktok_ads_catmum.json"
csv_path  = OUT_DIR / f"tiktok_ads_catmum.csv"

df.to_json(json_path, orient="records", force_ascii=False, indent=2)
df.to_csv(csv_path, index=False)

print(f"💾 Saved JSON: {json_path}")
print(f"💾 Saved CSV : {csv_path}")


🔎 Running query: sale


[apify.tiktok-ads-scraper runId:n9JTebPj54hwGzsFl] -> Status: RUNNING, Message: 
[apify.tiktok-ads-scraper runId:n9JTebPj54hwGzsFl] -> 2025-09-02T06:29:15.922Z ACTOR: Pulling Docker image of build XBM4iO1FpO4YIsl6f from registry.
[apify.tiktok-ads-scraper runId:n9JTebPj54hwGzsFl] -> 2025-09-02T06:29:15.929Z ACTOR: Creating Docker container.
[apify.tiktok-ads-scraper runId:n9JTebPj54hwGzsFl] -> 2025-09-02T06:29:15.996Z ACTOR: Starting Docker container.
[apify.tiktok-ads-scraper runId:n9JTebPj54hwGzsFl] -> 2025-09-02T06:29:16.373Z yarn run v1.22.19
[apify.tiktok-ads-scraper runId:n9JTebPj54hwGzsFl] -> 2025-09-02T06:29:16.398Z $ node dist/main.js --silent
[apify.tiktok-ads-scraper runId:n9JTebPj54hwGzsFl] -> 2025-09-02T06:29:16.843Z INFO  System info {"apifyVersion":"3.1.2","apifyClientVersion":"2.6.3","crawleeVersion":"3.3.0","osType":"Linux","nodeVersion":"v16.20.2"}
[apify.tiktok-ads-scraper runId:n9JTebPj54hwGzsFl] -> 2025-09-02T06:29:17.224Z INFO  CheerioCrawler: Status message: Craw


🔎 Running query: discount


[apify.tiktok-ads-scraper runId:2NDhozcVpCho4Ts5v] -> Status: RUNNING, Message: 
[apify.tiktok-ads-scraper runId:2NDhozcVpCho4Ts5v] -> 2025-09-02T06:29:32.721Z ACTOR: Pulling Docker image of build XBM4iO1FpO4YIsl6f from registry.
[apify.tiktok-ads-scraper runId:2NDhozcVpCho4Ts5v] -> 2025-09-02T06:29:32.734Z ACTOR: Creating Docker container.
[apify.tiktok-ads-scraper runId:2NDhozcVpCho4Ts5v] -> 2025-09-02T06:29:32.789Z ACTOR: Starting Docker container.
[apify.tiktok-ads-scraper runId:2NDhozcVpCho4Ts5v] -> 2025-09-02T06:29:33.167Z yarn run v1.22.19
[apify.tiktok-ads-scraper runId:2NDhozcVpCho4Ts5v] -> 2025-09-02T06:29:33.199Z $ node dist/main.js --silent
[apify.tiktok-ads-scraper runId:2NDhozcVpCho4Ts5v] -> 2025-09-02T06:29:33.694Z INFO  System info {"apifyVersion":"3.1.2","apifyClientVersion":"2.6.3","crawleeVersion":"3.3.0","osType":"Linux","nodeVersion":"v16.20.2"}
[apify.tiktok-ads-scraper runId:2NDhozcVpCho4Ts5v] -> 2025-09-02T06:29:34.172Z INFO  CheerioCrawler: Status message: Craw


🔎 Running query: offer


...


🔎 Running query: pro


[apify.tiktok-ads-scraper runId:396KaGyyDLy0xaxuK] -> Status: RUNNING, Message: 
[apify.tiktok-ads-scraper runId:396KaGyyDLy0xaxuK] -> 2025-09-02T06:57:03.824Z ACTOR: Pulling Docker image of build XBM4iO1FpO4YIsl6f from registry.
[apify.tiktok-ads-scraper runId:396KaGyyDLy0xaxuK] -> 2025-09-02T06:57:03.831Z ACTOR: Creating Docker container.
[apify.tiktok-ads-scraper runId:396KaGyyDLy0xaxuK] -> 2025-09-02T06:57:04.017Z ACTOR: Starting Docker container.
[apify.tiktok-ads-scraper runId:396KaGyyDLy0xaxuK] -> 2025-09-02T06:57:04.445Z yarn run v1.22.19
[apify.tiktok-ads-scraper runId:396KaGyyDLy0xaxuK] -> 2025-09-02T06:57:04.464Z $ node dist/main.js --silent
[apify.tiktok-ads-scraper runId:396KaGyyDLy0xaxuK] -> 2025-09-02T06:57:05.250Z INFO  System info {"apifyVersion":"3.1.2","apifyClientVersion":"2.6.3","crawleeVersion":"3.3.0","osType":"Linux","nodeVersion":"v16.20.2"}
[apify.tiktok-ads-scraper runId:396KaGyyDLy0xaxuK] -> 2025-09-02T06:57:05.575Z INFO  CheerioCrawler: Status message: Craw


🔎 Running query: max


[apify.tiktok-ads-scraper runId:jhZaZB6bFcROldxEd] -> Status: RUNNING, Message: 
[apify.tiktok-ads-scraper runId:jhZaZB6bFcROldxEd] -> 2025-09-02T06:57:32.220Z ACTOR: Pulling Docker image of build XBM4iO1FpO4YIsl6f from registry.
[apify.tiktok-ads-scraper runId:jhZaZB6bFcROldxEd] -> 2025-09-02T06:57:32.233Z ACTOR: Creating Docker container.
[apify.tiktok-ads-scraper runId:jhZaZB6bFcROldxEd] -> 2025-09-02T06:57:32.279Z ACTOR: Starting Docker container.
[apify.tiktok-ads-scraper runId:jhZaZB6bFcROldxEd] -> 2025-09-02T06:57:32.637Z yarn run v1.22.19
[apify.tiktok-ads-scraper runId:jhZaZB6bFcROldxEd] -> 2025-09-02T06:57:32.672Z $ node dist/main.js --silent
[apify.tiktok-ads-scraper runId:jhZaZB6bFcROldxEd] -> 2025-09-02T06:57:33.181Z INFO  System info {"apifyVersion":"3.1.2","apifyClientVersion":"2.6.3","crawleeVersion":"3.3.0","osType":"Linux","nodeVersion":"v16.20.2"}
[apify.tiktok-ads-scraper runId:jhZaZB6bFcROldxEd] -> 2025-09-02T06:57:33.579Z INFO  CheerioCrawler: Status message: Craw


✅ Collected 902 unique, valid ads (non-empty adTitle, no duplicate videos).
💾 Saved JSON: /content/drive/My Drive/FYP/tiktok_ads/tiktok_ads_catmum.json
💾 Saved CSV : /content/drive/My Drive/FYP/tiktok_ads/tiktok_ads_catmum.csv
